In [121]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [122]:
BATCH_SIZE = 32

## transformations
transform = transforms.Compose([transforms.ToTensor()])

## download and load training dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=4)

## download and load testing dataset
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=4)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 9912422/9912422 [00:00<00:00, 18862914.10it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 28881/28881 [00:00<00:00, 728600.69it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 1648877/1648877 [00:00<00:00, 6390189.66it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 4542/4542 [00:00<00:00, 8218519.74it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [134]:
print(len(trainset))
print(trainset[0])
first_image, first_label = trainset[0]
print(first_image)

60000
(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 

In [135]:
print(first_label)

5


In [136]:
first_image.shape

torch.Size([1, 28, 28])

In [142]:
for image, label in trainloader:
    print(f'image.shape = {image.shape}')
    print(f'label.shape = {label.shape}')
    break

image.shape = torch.Size([32, 1, 28, 28])
label.shape = torch.Size([32])


### output of a conv2d layer
Output = ((Input - Kernel_size + 2 * Padding) / Stride) + 1

stride=1 by default, padding=0 by default.

In [177]:
def get_cnn_output_shape(input_size, kernel_size, padding, stride):
    output = ((input_size - kernel_size + 2 * padding) / stride) + 1
    print(f'output size: {output}')
    return int(output)




Input Image:      [BATCHSIZE, IN_CHANNELS, HEIGHT, WIDTH]
                         ↓

Conv1 Filters:    32 filters of size [3, 3]
                         
                         ↓
Output Features:  [batch_size, 32, 26, 26]

In [178]:
IN_CHANNELS, HEIGHT, WIDTH = first_image.shape
print(IN_CHANNELS, HEIGHT, WIDTH)
print(BATCH_SIZE)
KERNEL_SIZE=3

1 28 28
32


In [179]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [180]:

class MNIST_CNN(nn.Module):
    def __init__(self, n_classes, height, batch_size, in_channels, out_channels, kernel_size, padding, stride, n_hidden):
        super(MNIST_CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size)
        
        self.output_size_conv1 = get_cnn_output_shape(input_size=height, kernel_size=kernel_size, padding=padding, stride=stride)

        self.linear1 = nn.Linear(self.output_size_conv1 * self.output_size_conv1 * out_channels, n_hidden)
        self.output_linear = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)
        #x = x.view(32, -1)

        # 32 x (32*26*26) => 32x128
        x = self.linear1(x)
        x = F.relu(x)

        # logits => 32x10
        logits = self.output_linear(x)
        out = F.softmax(logits, dim=1)
        return out

In [181]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
device

'cpu'

In [182]:
IN_CHANNELS, HEIGHT, WIDTH = first_image.shape
print(IN_CHANNELS, HEIGHT, WIDTH)
print(BATCH_SIZE)
KERNEL_SIZE=3
STRIDE=1
PADDING=0
N_CLASSES=10
N_HIDDEN = 16
CNN_model = MNIST_CNN(n_classes=N_CLASSES, 
                        height=HEIGHT, 
                        batch_size=BATCH_SIZE, 
                        in_channels=IN_CHANNELS,
                        out_channels=32,
                        kernel_size=KERNEL_SIZE, 
                        padding=PADDING, 
                        stride=STRIDE, 
                        n_hidden=N_HIDDEN)

CNN_model = CNN_model.to(device)

1 28 28
32
output size: 26.0


In [185]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNN_model.parameters(), lr=1E-3)

In [186]:
EPOCHS=2
for epoch in range(EPOCHS):
    train_running_loss = 0.0
    train_acc = 0.0

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = CNN_model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits, 1).flatten() == labels).type(torch.float).mean().item()
    
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))

Epoch: 0 | Loss: 1.5875 | Train Accuracy: 0.88
Epoch: 1 | Loss: 1.5054 | Train Accuracy: 0.96


In [ ]:
#conda install anaconda::networkx
#conda install conda-forge::tensorboardx

In [187]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [188]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
